In [1]:
import pandas as pd 

dataset = pd.read_csv('./dataset/train.csv')
dataset.head()

ID  target        v1        v2 v3        v4         v5        v6        v7  \
0   3       1  1.335739  8.727474  C  3.921026   7.915266  2.599278  3.176895   
1   4       1       NaN       NaN  C       NaN   9.191265       NaN       NaN   
2   5       1  0.943877  5.310079  C  4.410969   5.326159  3.979592  3.928571   
3   6       1  0.797415  8.304757  C  4.225930  11.627438  2.097700  1.987549   
4   8       1       NaN       NaN  C       NaN        NaN       NaN       NaN   

         v8    ...         v122      v123      v124  v125      v126      v127  \
0  0.012941    ...     8.000000  1.989780  0.035754    AU  1.804126  3.113719   
1  2.301630    ...          NaN       NaN  0.598896    AF       NaN       NaN   
2  0.019645    ...     9.333333  2.477596  0.013452    AE  1.773709  3.922193   
3  0.171947    ...     7.018256  1.812795  0.002267    CJ  1.415230  2.954381   
4       NaN    ...          NaN       NaN       NaN     Z       NaN       NaN   

       v128  v129      v130      v131  
0  2.024285     0  0.636365  2.857144  
1  1.957825     0       NaN       NaN  
2  1.120468     2  0.883118  1.176472  
3  1.990847     1  1.677108  1.034483  
4       NaN     0       NaN       NaN  

[5 rows x 133 columns]

In [2]:
[col for col in dataset.columns if dataset[col].isnull().sum()>0]

['v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v10',
 'v11',
 'v12',
 'v13',
 'v14',
 'v15',
 'v16',
 'v17',
 'v18',
 'v19',
 'v20',
 'v21',
 'v22',
 'v23',
 'v25',
 'v26',
 'v27',
 'v28',
 'v29',
 'v30',
 'v31',
 'v32',
 'v33',
 'v34',
 'v35',
 'v36',
 'v37',
 'v39',
 'v40',
 'v41',
 'v42',
 'v43',
 'v44',
 'v45',
 'v46',
 'v48',
 'v49',
 'v50',
 'v51',
 'v52',
 'v53',
 'v54',
 'v55',
 'v56',
 'v57',
 'v58',
 'v59',
 'v60',
 'v61',
 'v63',
 'v64',
 'v65',
 'v67',
 'v68',
 'v69',
 'v70',
 'v73',
 'v76',
 'v77',
 'v78',
 'v80',
 'v81',
 'v82',
 'v83',
 'v84',
 'v85',
 'v86',
 'v87',
 'v88',
 'v89',
 'v90',
 'v91',
 'v92',
 'v93',
 'v94',
 'v95',
 'v96',
 'v97',
 'v98',
 'v99',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v106',
 'v107',
 'v108',
 'v109',
 'v111',
 'v112',
 'v113',
 'v114',
 'v115',
 'v116',
 'v117',
 'v118',
 'v119',
 'v120',
 'v121',
 'v122',
 'v123',
 'v124',
 'v125',
 'v126',
 'v127',
 'v128',
 'v130',
 'v131']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.drop(labels=['ID', 'target'], axis=1), dataset['target'],
                                                                                            test_size=.3, random_state=42)

X_train.shape, X_test.shape

((80024, 131), (34297, 131))

In [5]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(dataset.select_dtypes(include=numerics).columns)
dataset = dataset[numerical_vars]
dataset.shape

(114321, 114)

In [6]:
corrmat = X_train.corr()
corrmat = corrmat.abs().unstack()
corrmat = corrmat.sort_values(ascending=False)
corrmat = corrmat[corrmat >= 0.8]
corrmat = corrmat[corrmat <1 ]
corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
corrmat.head()

feature1 feature2      corr
0      v58     v100  0.997622
1     v100      v58  0.997622
2      v69     v115  0.993573
3     v115      v69  0.993573
4      v76      v17  0.992519

In [17]:
grouped_feature_ls = []
correlated_groups = []

for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:
        correlated_block = corrmat[corrmat.feature1 == feature]
        grouped_feature_ls = grouped_feature_ls + list(correlated_block.feature2.unique()) + [feature]
        correlated_groups.append(correlated_block)

print('found {} correlated groups'.format(len(correlated_groups)))
print('out of {} total features'.format(X_train.shape[1]))

found 31 correlated groups
out of 131 total features


In [19]:
for group in correlated_groups:
    print(group)
    print('///////////////////////////////')

  feature1 feature2      corr
0      v58     v100  0.997622
///////////////////////////////
  feature1 feature2      corr
2      v69     v115  0.993573
///////////////////////////////
    feature1 feature2      corr
4        v76      v17  0.992519
51       v76      v64  0.953521
146      v76      v93  0.874306
163      v76      v48  0.864467
///////////////////////////////
   feature1 feature2      corr
6       v46      v63  0.991654
11      v46      v25  0.982290
24      v46       v8  0.975250
59      v46     v105  0.946311
75      v46      v89  0.932277
88      v46      v54  0.919460
///////////////////////////////
    feature1 feature2      corr
14       v60      v26  0.980771
144      v60      v43  0.876529
174      v60     v116  0.860019
///////////////////////////////
    feature1 feature2      corr
17      v106      v48  0.979785
73      v106      v64  0.933384
148     v106      v44  0.873250
168     v106      v61  0.863056
179     v106      v17  0.857544
186     v106       v4  

In [20]:
group = correlated_groups[2]
group 

feature1 feature2      corr
4        v76      v17  0.992519
51       v76      v64  0.953521
146      v76      v93  0.874306
163      v76      v48  0.864467

In [21]:
from sklearn.ensemble import RandomForestClassifier

features = list(group.feature2.unique())+['v17']
rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
rf.fit(X_train[features].fillna(0), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=39, verbose=0, warm_start=False)

In [22]:
importance = pd.concat(
    [pd.Series(features),
     pd.Series(rf.feature_importances_)], axis=1)

importance.columns = ['feature', 'importance']
importance.sort_values(by='importance', ascending=False)

feature  importance
3     v48    0.375427
2     v93    0.166766
0     v17    0.155322
1     v64    0.151751
4     v17    0.150734